# Data Cleaning
`Dataset:` housing_data_2016_2017.csv 

This dataset is the raw data representation of Housing data in 2016 and 2017 found at MLSI. The dataset was harvested with Amazon's MTurk and it is a raw download from their system.


- I.   Data Exploration
- II.  Scope Definition 
- III. Removal of Unnecessary features
- IV.  Converting Addresses to zones


In [606]:
import pandas as pd
import numpy as np
import os 
import matplotlib.pyplot as plt
import seaborn as sns
%pip install missingno
%pip install sklearn.preprocessing import LabelEncoder

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



  Using cached sklearn_preprocessing-0.1.0-py3-none-any.whl.metadata (70 bytes)


ERROR: Could not find a version that satisfies the requirement import (from versions: none)
ERROR: No matching distribution found for import

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [607]:
housing_data = pd.read_csv("housing_data_2016_2017.csv")

# (2,230 Rows/Entries , 55 Columns/Features)
housing_data.shape

(2230, 55)

In [608]:
housing_data.describe()

,Keywords,MaxAssignments,AssignmentDurationInSeconds,AutoApprovalDelayInSeconds,NumberOfSimilarHITs,LifetimeInSeconds,RejectionTime,RequesterFeedback,WorkTimeInSeconds,approx_year_built,community_district_num,num_bedrooms,num_floors_in_building,num_full_bathrooms,num_half_bathrooms,num_total_rooms,pct_tax_deductibl,sq_footage,walk_score
count,0.0,1472.0,1472.0,1472.0,0.0,0.0,0.0,0.0,1472.000000,2190.000000,2211.000000,2115.000000,1580.000000,2230.000000,172.000000,2228.000000,476.00000,1020.000000,2230.000000
mean,NaN,1.0,900.0,60.0,NaN,NaN,NaN,NaN,162.389946,1962.712329,26.326549,1.653428,7.785443,1.231390,0.953488,4.138689,45.39916,955.361765,83.915695
std,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,111.694958,21.079921,2.953197,0.743614,7.515838,0.444599,0.302310,1.345909,6.94886,380.864722,14.747313
min,NaN,1.0,900.0,60.0,NaN,NaN,NaN,NaN,22.000000,1893.000000,3.000000,0.000000,1.000000,1.000000,0.000000,0.000000,20.00000,100.000000,7.000000
25%,NaN,1.0,900.0,60.0,NaN,NaN,NaN,NaN,89.000000,1950.000000,25.000000,1.000000,3.000000,1.000000,1.000000,3.000000,40.00000,743.000000,77.000000
50%,NaN,1.0,900.0,60.0,NaN,NaN,NaN,NaN,127.000000,1958.000000,26.000000,2.000000,6.000000,1.000000,1.000000,4.000000,50.00000,881.000000,89.000000
75%,NaN,1.0,900.0,60.0,NaN,NaN,NaN,NaN,197.000000,1970.000000,28.000000,2.000000,7.000000,1.000000,1.000000,5.000000,50.00000,1100.000000,95.000000
max,NaN,1.0,900.0,60.0,NaN,NaN,NaN,NaN,815.000000,2017.000000,32.000000,6.000000,34.000000,3.000000,2.000000,14.000000,75.00000,6215.000000,99.000000


In [609]:
housing_data.columns

Index(['HITId', 'HITTypeId', 'Title', 'Description', 'Keywords', 'Reward',
       'CreationTime', 'MaxAssignments', 'RequesterAnnotation',
       'AssignmentDurationInSeconds', 'AutoApprovalDelayInSeconds',
       'Expiration', 'NumberOfSimilarHITs', 'LifetimeInSeconds',
       'AssignmentId', 'WorkerId', 'AssignmentStatus', 'AcceptTime',
       'SubmitTime', 'AutoApprovalTime', 'ApprovalTime', 'RejectionTime',
       'RequesterFeedback', 'WorkTimeInSeconds', 'LifetimeApprovalRate',
       'Last30DaysApprovalRate', 'Last7DaysApprovalRate', 'URL',
       'approx_year_built', 'cats_allowed', 'common_charges',
       'community_district_num', 'coop_condo', 'date_of_sale',
       'dining_room_type', 'dogs_allowed', 'fuel_type',
       'full_address_or_zip_code', 'garage_exists', 'kitchen_type',
       'maintenance_cost', 'model_type', 'num_bedrooms',
       'num_floors_in_building', 'num_full_bathrooms', 'num_half_bathrooms',
       'num_total_rooms', 'parking_charges', 'pct_tax_deductibl'

# I. Scope
The limitation on the data population to predict on:

`LOCATION` : Queens, NY <br>
`HOME TYPES` :  Condo / homeowner assoc / and Co-op <br> 
`MAXIMUM SALE PRICE` :  >= $1M <br>
`TIME FRAME` :  February 2016 to February 2017 <br>

`ZIP CODES` : 
- `Northeast Queens` 11361 11362, 11363 11364
- `North Queens` 11354 11355 11356 11357 11358 11359 11360
- `Central Queens` 11365 11366 11367
- `Jamaica` 11412 11423 11432 11433 11434 11435 11436
- `Northwest Queens` 11101 11102 11103 11104 11105 11106
- `West Central Queens` 11374 11375 11379 11385
- `Southeast Queens` 11004 11005 11411 11413 11422 11426 11427 11428 11429
- `Southwest Queens` 11414 11415 11416 11417 11418 11419 11420 11421
- `West Queens` 11368 11369 11370 11372 11373 11377 11378

## II. Removal of Unnecessary Features


In [610]:
# Drop Unusable Columns
pd.set_option('display.max_columns', None)
print(housing_data.head())

unusable_features = [
                    "HITId", "HITTypeId", "Title", "Description", "Keywords", "Reward", "CreationTime", "MaxAssignments", "RequesterAnnotation", "AssignmentDurationInSeconds", "AutoApprovalDelayInSeconds",
                     "Expiration", "NumberOfSimilarHITs", "LifetimeInSeconds", "AssignmentId", "WorkerId", "AssignmentStatus", "AcceptTime", "SubmitTime", "AutoApprovalTime", "ApprovalTime", "RejectionTime",
                     "RequesterFeedback", "WorkTimeInSeconds", "LifetimeApprovalRate", "Last30DaysApprovalRate", "Last7DaysApprovalRate", "URL", "pct_tax_deductibl", "total_taxes", "listing_price_to_nearest_1000",
                     "url"
                     ]

useful_features = [
                   "approx_year_built", "cats_allowed", "common_charges", "community_district_num", "coop_condo", "date_of_sale", "dining_room_type", "dogs_allowed", "fuel_type", "full_address_or_zip_code",
                   "garage_exists", "kitchen_type", "maintenance_cost", "model_type", "num_bedrooms", "num_floors_in_building", "num_full_bathrooms", "num_half_bathrooms", "num_total_rooms", 
                   "parking_charges", "sale_price", "sq_footage", "walk_score", 
                    ]


                            HITId                       HITTypeId  \
0  3OID399FXG7F26JWONXF0Y86J90FD4  36BILMLQB75QQNBTYKGYCZWDN8TVAU   
1  3MQY1YVHS3K2MF90MWR2LPQH7KJ2B0  36BILMLQB75QQNBTYKGYCZWDN8TVAU   
2  3DGDV62G7O94Q9AA5193G9V6OOY2PL  36BILMLQB75QQNBTYKGYCZWDN8TVAU   
3  3087LXLJ6MGL3MI2CB9KLRONPKRF0B  36BILMLQB75QQNBTYKGYCZWDN8TVAU   
4  3FULMHZ7OUX88KSKHZ0ZSKY93XJ4MN  36BILMLQB75QQNBTYKGYCZWDN8TVAU   

                                               Title  \
0  Find Information about Housing To Help a Stude...   
1  Find Information about Housing To Help a Stude...   
2  Find Information about Housing To Help a Stude...   
3  Find Information about Housing To Help a Stude...   
4  Find Information about Housing To Help a Stude...   

                                      Description  Keywords  Reward  \
0  Go to a link and copy information into the HIT       NaN  $0.05    
1  Go to a link and copy information into the HIT       NaN  $0.05    
2  Go to a link and copy informatio

In [611]:
print("Total number of columns: ", len(housing_data.columns))
print("Total number of unusable features: ", len(unusable_features))
print("Total number of usable features:", len(useful_features))
print("Unusable + Usable = ", len(unusable_features) + len(useful_features))

Total number of columns:  55
Total number of unusable features:  32
Total number of usable features: 23
Unusable + Usable =  55


## III. Feature Engineering
- Conversion of data types
- Imputation
- Converting leveled categorical to numerical while maintaining levels
- conversion of some floats to ints for faster computation 
- readjustment of columns as groups 

In [612]:
# Drop unusable columns 
housing_data_reduced = housing_data.drop(columns=unusable_features)
housing_data_reduced

,approx_year_built,cats_allowed,common_charges,community_district_num,coop_condo,date_of_sale,dining_room_type,dogs_allowed,fuel_type,full_address_or_zip_code,garage_exists,kitchen_type,maintenance_cost,model_type,num_bedrooms,num_floors_in_building,num_full_bathrooms,num_half_bathrooms,num_total_rooms,parking_charges,sale_price,sq_footage,walk_score
0,1955.0,no,$767,25.0,co-op,2/16/2016,combo,no,gas,"Flushing NY, 11355",NaN,eat in,NaN,Mitchell Garden 3,2.0,6.0,1,NaN,5.0,NaN,"$228,000",NaN,82
1,1955.0,no,NaN,25.0,co-op,2/16/2016,formal,no,oil,"30-11 Parsons Blvd, Flushing NY, 11354 ( Sold...",NaN,eat in,$604,Jr-4 Model,1.0,7.0,1,NaN,4.0,NaN,"$235,500",890.0,89
2,2004.0,no,$167,24.0,condo,2/17/2016,combo,no,NaN,"102-14 Lewis Ave, Corona NY, 11368",NaN,efficiency,NaN,Apt In Bldg,1.0,1.0,1,NaN,3.0,NaN,"$137,550",550.0,90
3,2002.0,no,$275,25.0,condo,2/17/2016,combo,no,gas,"144-48 Roosevelt Ave, Flushing NY, 11354",NaN,eat in,NaN,144-48 Roosevelt,3.0,NaN,2,NaN,5.0,NaN,"$545,000",NaN,94
4,1949.0,yes,NaN,26.0,co-op,2/18/2016,combo,yes,gas,"245-27 76th Ave, Bellerose NY, 11426",NaN,eat in,$660,C-1,2.0,2.0,1,NaN,4.0,NaN,"$241,700",675.0,71
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2225,1987.0,no,$480,25.0,condo,NaN,combo,no,gas,"Not AvailableFlushing NY, 11355",NaN,combo,NaN,Colden Luxury Condo,2.0,7.0,1,NaN,4.0,NaN,NaN,NaN,97
2226,1983.0,yes,$956,25.0,condo,NaN,formal,no,gas,"One Bay Club Dr, Bayside NY, 11360",yes,eatin,NaN,2 Br Deluxe,2.0,NaN,2,NaN,5.0,$99,NaN,NaN,82
2227,2010.0,no,$250,24.0,condo,NaN,formal,no,gas,"Ridgewood NY, 11385",NaN,combo,NaN,Modern,3.0,NaN,2,NaN,6.0,NaN,NaN,1500.0,96
2228,2010.0,no,$250,24.0,condo,NaN,formal,no,gas,"Ridgewood NY, 11385",NaN,combo,NaN,Condo,3.0,4.0,2,NaN,6.0,NaN,NaN,1600.0,96


In [613]:
# Convert date to date_time object 
print("Converting date to date object ...")
housing_data_reduced['date_of_sale'] = pd.to_datetime(housing_data['date_of_sale'])

Converting date to date object ...


In [614]:
print("One hot encoding of cats_allowed...")

print("Before: " , housing_data_reduced['cats_allowed'].unique())
housing_data_reduced['cats_allowed'] = housing_data_reduced['cats_allowed'].replace({'y':'yes'}) # Convert y -> yes
housing_data_reduced['cats_allowed'] = housing_data_reduced['cats_allowed'].map({'yes': 1, 
                                                                                 'no': 0})
print("After: " , housing_data_reduced['cats_allowed'].unique())


One hot encoding of cats_allowed...
Before:  ['no' 'yes' 'y']
After:  [0 1]


In [615]:
print("Conversion of ordinal categorical into leveled factors...")
# 1 -> Condo
# 2 -> Co-op
print("Before: " , housing_data_reduced['coop_condo'].unique())
# Scale Map to preserve order
coop_condo_scaler = {'co-op' : 1,
                     'condo' : 2}
# Last element of the categories array has the highest priority
housing_data_reduced['coop_condo'] = housing_data_reduced['coop_condo'].replace(coop_condo_scaler)
print("After: " , housing_data_reduced['coop_condo'].unique())

Conversion of ordinal categorical into leveled factors...
Before:  ['co-op' 'condo']
After:  [1 2]


In [616]:
print("Conversion of Sale Price from string to numeric and the removal of dollar sign...")

# Before
print("Before :")
print(housing_data_reduced['sale_price'])

#   Remove currency symbols and commas
housing_data_reduced['sale_price'] = housing_data_reduced['sale_price'].str.replace('[\$,]', '', regex=True)
# convert to numeric, coercing errors to NaN
housing_data_reduced['sale_price'] = pd.to_numeric(housing_data_reduced['sale_price'], errors='coerce')

# After
print("\nSale_price values after conversion to numeric:")
print(housing_data_reduced['sale_price'])

Conversion of Sale Price from string to numeric and the removal of dollar sign...
Before :
0       $228,000 
1       $235,500 
2       $137,550 
3       $545,000 
4       $241,700 
          ...    
2225          NaN
2226          NaN
2227          NaN
2228          NaN
2229          NaN
Name: sale_price, Length: 2230, dtype: object

Sale_price values after conversion to numeric:
0       228000.0
1       235500.0
2       137550.0
3       545000.0
4       241700.0
          ...   
2225         NaN
2226         NaN
2227         NaN
2228         NaN
2229         NaN
Name: sale_price, Length: 2230, dtype: float64


In [617]:
print("Finding the mean sale price per dining_room_type...")
# Find the mean price of each dining_room_type to find which one ranks the highest
# Take a subset with values on sale_price so we can get the mean
housing_data_reduced_subset = housing_data_reduced.dropna(subset=['sale_price'])
# Group by 'dining_room_type' and calculate the mean of 'sale_price'
mean_sale_price_by_dining_room = housing_data_reduced_subset.groupby('dining_room_type')['sale_price'].mean()

Finding the mean sale price per dining_room_type...


In [618]:
# Results for dining_room_type
print("Mean sale_price by dining_room_type:")
print(mean_sale_price_by_dining_room)

# Create a scaler to maintain order based off of mean price
dining_room_type_scaler = { 'none' : 1,
                     'dining area' : 2,
                     'combo' : 3,
                     'other' : 4,
                     'formal' : 5}

Mean sale_price by dining_room_type:
dining_room_type
combo          298695.763485
dining area    177750.000000
formal         365289.646552
other          300683.673469
Name: sale_price, dtype: float64


In [619]:
# Convert to numeric factor
print("Converting dining_room_type from categorical to numeric...")
print("Before :" , housing_data_reduced['dining_room_type'].unique())
# Conversion
housing_data_reduced['dining_room_type'] = housing_data_reduced['dining_room_type'].replace(dining_room_type_scaler)
# Cast float to int to remove decimal point (ONLY TO non-NA values)
print("After :" , housing_data_reduced['dining_room_type'].unique())

Converting dining_room_type from categorical to numeric...
Before : ['combo' 'formal' nan 'other' 'dining area' 'none']
After : [ 3.  5. nan  4.  2.  1.]


In [620]:
print("One hot encoding of dogs_allowed...")

print("Before: " , housing_data_reduced['dogs_allowed'].unique())
housing_data_reduced['dogs_allowed'] = housing_data_reduced['dogs_allowed'].replace({'yes89':'yes'}) # Convert y -> yes
housing_data_reduced['dogs_allowed'] = housing_data_reduced['dogs_allowed'].map({'yes': 1, 
                                                                                  'no': 0})
print("After: " , housing_data_reduced['dogs_allowed'].unique())

One hot encoding of dogs_allowed...
Before:  ['no' 'yes' 'yes89']
After:  [0 1]


In [621]:
print("Finding mean sale_price per fuel_type...")
# Find the mean price of each kitchen_type to find which one ranks the highest

# Join other && Other
housing_data_reduced['fuel_type'] = housing_data_reduced['fuel_type'].replace({'Other':'other'})
housing_data_reduced['fuel_type'].unique()

# Take a subset with values of sale_price so we can get the mean
# Group by 'dining_room_type' and calculate the mean of 'sale_price'
mean_sale_price_by_fuel_type = housing_data_reduced_subset.groupby('fuel_type')['sale_price'].mean()
print("Mean_sale_price_by_fuel_type: \n", mean_sale_price_by_fuel_type)

# Create a scaler to maintain order based off of mean price
fuel_type_scaler = { 'oil' : 1,
                     'none' : 2,
                     'gas' : 3,
                     'other' : 4,
                     'Other' : 4,
                     'electric' : 5}


Finding mean sale_price per fuel_type...
Mean_sale_price_by_fuel_type: 
 fuel_type
Other       430000.000000
electric    438454.545455
gas         336916.990033
none        298333.333333
oil         264277.783333
other       430562.500000
Name: sale_price, dtype: float64


In [622]:
# Convert to numeric factor
print("Converting fuel_type from categorical to numeric...")
print("Before :" , housing_data_reduced['fuel_type'].unique())
# Conversion
housing_data_reduced['fuel_type'] = housing_data_reduced['fuel_type'].replace(fuel_type_scaler)
# Cast float to int to remove decimal point (ONLY TO non-NA values)
print("After :" , housing_data_reduced['fuel_type'].unique())


Converting fuel_type from categorical to numeric...
Before : ['gas' 'oil' nan 'other' 'electric' 'none']
After : [ 3.  1. nan  4.  5.  2.]


In [623]:
print("Encoding garage_exists...")
# Assume that NaN's don't have a garage
print("Before :", housing_data_reduced['garage_exists'].unique())

# If NaN = 0, Else = 1
#Set all null values to 0
housing_data_reduced['garage_exists'].fillna(0, inplace=True)
# Set all non-null values to 1
housing_data_reduced['garage_exists'] = housing_data_reduced['garage_exists'].apply(lambda x: 1 if x != 0 else 0)
print("After :", housing_data_reduced['garage_exists'].unique())

Encoding garage_exists...
Before : [nan 'Underground' 'Yes' 'yes' 'UG' '1' 'eys']
After : [0 1]


In [624]:
# Find the mean price of each kitchen_type to find which one ranks the highest
print("Finding mean sale_price per kitchen_type...")
print("Before: ", housing_data_reduced['kitchen_type'].unique())
print()


# Normalize kitchen_type

# Store variations in arrays
efficiency = ["efficiencykitchen", "efficiencyktchen", "efficiemcy", "efficiencykitchen", "efficiencykitchene", "efficiency", "efficiency kitchene", "efficiency kitchen", "efficiency ktchen"]
eatin = ["eatin", "EatIn", "Eatin", "eatin", "eat in", "Eat in", "Eat In"]
combo = ["combo", "Combo"]
other = ["1955", 'other']

# Define a function to clean the values
def clean_kitchen_type(value):
    if pd.isna(value):
        return value  # Keep NaN as is
    value = value.lower().strip()  # Normalize case and whitespace
    if value in efficiency:
        return 'efficiency'
    elif value in eatin:
        return 'eatin'
    elif value in combo:
        return 'combo'
    elif value in other:
        return 'other'
    else:
        return value

# Apply the function to the DataFrame column
housing_data_reduced['kitchen_type'] = housing_data_reduced['kitchen_type'].apply(clean_kitchen_type)

# Group by 'kitchen_type' and calculate the mean of 'sale_price'
mean_sale_price_by_kitchen_type = housing_data_reduced.groupby('kitchen_type')['sale_price'].mean()

print(mean_sale_price_by_kitchen_type)
print()
print("After: ", housing_data_reduced['kitchen_type'].unique())

Finding mean sale_price per kitchen_type...
Before:  ['eat in' 'efficiency' 'Combo' 'combo' 'Eat In' nan 'Eat in' '1955'
 'eatin' 'efficiency kitchene' 'efficiency kitchen' 'efficiemcy' 'none'
 'efficiency ktchen']

kitchen_type
combo         373358.962963
eatin         346329.779904
efficiency    265095.519481
none                    NaN
other         325000.000000
Name: sale_price, dtype: float64

After:  ['eatin' 'efficiency' 'combo' nan 'other' 'none']


In [625]:
kitchen_type_scaler = { 'none' : 1,
                     'efficiency' : 2,
                     'other' : 3,
                     'eatin' : 4,
                     'combo' : 5,
                    }


# Convert to numeric factor
print("Converting fuel_type from categorical to numeric...")
print("Before :" , housing_data_reduced['kitchen_type'].unique())
# Conversion
housing_data_reduced['kitchen_type'] = housing_data_reduced['kitchen_type'].replace(kitchen_type_scaler)
# Cast float to int to remove decimal point (ONLY TO non-NA values)
print("After :" , housing_data_reduced['kitchen_type'].unique())


Converting fuel_type from categorical to numeric...
Before : ['eatin' 'efficiency' 'combo' nan 'other' 'none']
After : [ 4.  2.  5. nan  3.  1.]


In [626]:
print("Filling in num_bedroom NaN entries with 0 ...")
print("Before:", housing_data_reduced['num_bedrooms'].isna().sum())
housing_data_reduced['num_bedrooms'] = housing_data_reduced['num_bedrooms'].fillna(0)
print("After:", housing_data_reduced['num_bedrooms'].isna().sum())


Filling in num_bedroom NaN entries with 0 ...
Before: 115
After: 0


In [627]:
print("Filling in num_half_bathrooms NaN entries with 0 ...")
print("Before:", housing_data_reduced['num_half_bathrooms'].isna().sum())
housing_data_reduced['num_half_bathrooms'] = housing_data_reduced['num_half_bathrooms'].fillna(0)
print("After:", housing_data_reduced['num_half_bathrooms'].isna().sum())


Filling in num_half_bathrooms NaN entries with 0 ...
Before: 2058
After: 0


In [628]:
print("Filling in num_total_rooms NaN entries with 0 ...")
print("Before:", housing_data_reduced['num_total_rooms'].isna().sum())
housing_data_reduced['num_total_rooms'] = housing_data_reduced['num_total_rooms'].fillna(0)
print("After:", housing_data_reduced['num_total_rooms'].isna().sum())


Filling in num_total_rooms NaN entries with 0 ...
Before: 2
After: 0


In [629]:
print("Filling in parking_charges NaN entries with 0 ...")
print("Before:", housing_data_reduced['parking_charges'].isna().sum())
housing_data_reduced['parking_charges'] = housing_data_reduced['parking_charges'].fillna(0)
print("After:", housing_data_reduced['parking_charges'].isna().sum())


Filling in parking_charges NaN entries with 0 ...
Before: 1671
After: 0


In [630]:
print("Formatting and data type conversion of maintenance_cost and parking_charges")

print("maintenance_cost data type before: ", housing_data_reduced["maintenance_cost"].dtypes)
print("parking_charges data type before: ", housing_data_reduced["parking_charges"].dtypes)

# Function to convert dollar amount strings to integer, ignoring NaNs
def convert_dollar_to_int(dollar_str):
    if pd.isna(dollar_str):
        return np.nan
    if isinstance(dollar_str, str):
        return int(dollar_str.replace('$', '').replace(',', ''))
    else:
        return int(dollar_str)

# Apply the function to the specific columns
housing_data_reduced['maintenance_cost'] = housing_data_reduced['maintenance_cost'].apply(convert_dollar_to_int)
housing_data_reduced['parking_charges'] = housing_data_reduced['parking_charges'].apply(convert_dollar_to_int)

# Display the updated DataFrame
print("maintenance_cost data type after: ", housing_data_reduced["maintenance_cost"].dtypes)
print("parking_charges data type after: ", housing_data_reduced["parking_charges"].dtypes)

Formatting and data type conversion of maintenance_cost and parking_charges
maintenance_cost data type before:  object
parking_charges data type before:  object
maintenance_cost data type after:  float64
parking_charges data type after:  int64


In [631]:
print("Converting data types...")
print("Before: ")
colnames = {}
for column in housing_data_reduced.columns:
    colnames[column] = housing_data_reduced[column].dtype

for col in colnames.keys():
    print(f"{col}: ", colnames[col])

print("After: ")
# List of columns to convert to integers
columns_to_convert = [
    'garage_exists', 'num_bedrooms', 'num_floors_in_building', 'num_full_bathrooms', 
    'num_half_bathrooms', 'num_total_rooms', 'dining_room_type', 'fuel_type', 
    'kitchen_type', 'community_district_num', 'approx_year_built', 'common_charges', "approx_year_built"
]

# Function to safely convert to int
def safe_int_convert(value):
    try:
        return int(value)
    except (ValueError, TypeError):
        return np.nan

# Convert specified columns to integers
for column in columns_to_convert:
    housing_data_reduced[column] = housing_data_reduced[column].apply(safe_int_convert)

# Display the updated DataFrame data types
print(housing_data_reduced.dtypes)

Converting data types...
Before: 
approx_year_built:  float64
cats_allowed:  int64
common_charges:  object
community_district_num:  float64
coop_condo:  int64
date_of_sale:  datetime64[ns]
dining_room_type:  float64
dogs_allowed:  int64
fuel_type:  float64
full_address_or_zip_code:  object
garage_exists:  int64
kitchen_type:  float64
maintenance_cost:  float64
model_type:  object
num_bedrooms:  float64
num_floors_in_building:  float64
num_full_bathrooms:  int64
num_half_bathrooms:  float64
num_total_rooms:  float64
parking_charges:  int64
sale_price:  float64
sq_footage:  float64
walk_score:  int64
After: 
approx_year_built                  float64
cats_allowed                         int64
common_charges                     float64
community_district_num             float64
coop_condo                           int64
date_of_sale                datetime64[ns]
dining_room_type                   float64
dogs_allowed                         int64
fuel_type                          float64

In [632]:
# Assuming df is your original DataFrame
# Define the desired column order
df_order = [
    'full_address_or_zip_code', 'model_type', 'date_of_sale',
    'cats_allowed', 'dogs_allowed', 'coop_condo', 'garage_exists',
    'num_bedrooms', 'num_floors_in_building', 'num_full_bathrooms',  'num_half_bathrooms', 'num_total_rooms',
    'dining_room_type', 'fuel_type', 'kitchen_type', 
    'community_district_num',
    'approx_year_built', 'sq_footage', 'walk_score',
    'common_charges', 'maintenance_cost', 'parking_charges',
    'sale_price'
]

# Reorder the DataFrame columns
housing_data_reduced = housing_data_reduced[df_order]

# Display the rearranged DataFrame
housing_data_reduced

,full_address_or_zip_code,model_type,date_of_sale,cats_allowed,dogs_allowed,coop_condo,garage_exists,num_bedrooms,num_floors_in_building,num_full_bathrooms,num_half_bathrooms,num_total_rooms,dining_room_type,fuel_type,kitchen_type,community_district_num,approx_year_built,sq_footage,walk_score,common_charges,maintenance_cost,parking_charges,sale_price
0,"Flushing NY, 11355",Mitchell Garden 3,2016-02-16,0,0,1,0,2,6.0,1,0,5,3.0,3.0,4.0,25.0,1955.0,NaN,82,NaN,NaN,0,228000.0
1,"30-11 Parsons Blvd, Flushing NY, 11354 ( Sold...",Jr-4 Model,2016-02-16,0,0,1,0,1,7.0,1,0,4,5.0,1.0,4.0,25.0,1955.0,890.0,89,NaN,604.0,0,235500.0
2,"102-14 Lewis Ave, Corona NY, 11368",Apt In Bldg,2016-02-17,0,0,2,0,1,1.0,1,0,3,3.0,NaN,2.0,24.0,2004.0,550.0,90,NaN,NaN,0,137550.0
3,"144-48 Roosevelt Ave, Flushing NY, 11354",144-48 Roosevelt,2016-02-17,0,0,2,0,3,NaN,2,0,5,3.0,3.0,4.0,25.0,2002.0,NaN,94,NaN,NaN,0,545000.0
4,"245-27 76th Ave, Bellerose NY, 11426",C-1,2016-02-18,1,1,1,0,2,2.0,1,0,4,3.0,3.0,4.0,26.0,1949.0,675.0,71,NaN,660.0,0,241700.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2225,"Not AvailableFlushing NY, 11355",Colden Luxury Condo,NaT,0,0,2,0,2,7.0,1,0,4,3.0,3.0,5.0,25.0,1987.0,NaN,97,NaN,NaN,0,NaN
2226,"One Bay Club Dr, Bayside NY, 11360",2 Br Deluxe,NaT,1,0,2,1,2,NaN,2,0,5,5.0,3.0,4.0,25.0,1983.0,NaN,82,NaN,NaN,99,NaN
2227,"Ridgewood NY, 11385",Modern,NaT,0,0,2,0,3,NaN,2,0,6,5.0,3.0,5.0,24.0,2010.0,1500.0,96,NaN,NaN,0,NaN
2228,"Ridgewood NY, 11385",Condo,NaT,0,0,2,0,3,4.0,2,0,6,5.0,3.0,5.0,24.0,2010.0,1600.0,96,NaN,NaN,0,NaN


# IV. Conversion of address into zones

## ZONES: 
`ZIP CODES` : 
1. `Northeast Queens` 11361 11362, 11363 11364
2. `North Queens` 11354 11355 11356 11357 11358 11359 11360
3. `Central Queens` 11365 11366 11367
4. `Jamaica` 11412 11423 11432 11433 11434 11435 11436
5. `Northwest Queens` 11101 11102 11103 11104 11105 11106
6. `West Central Queens` 11374 11375 11379 11385
7. `Southeast Queens` 11004 11005 11411 11413 11422 11426 11427 11428 11429
8.  `Southwest Queens` 11414 11415 11416 11417 11418 11419 11420 11421
9. `West Queens` 11368 11369 11370 11372 11373 11377 11378

## EXTRACT ZIPCODE TO A NEW COLUMN

In [633]:
import re  # Import regular expression module

# Function to extract ZIP code
def extract_zipcode(address):
    if isinstance(address, str):
        match = re.search(r'\b\d{5}\b', address.strip())
        return match.group(0) if match else None
    return None

# Extract ZIP codes and identify addresses without ZIP codes
zipcodes = []
addresses_without_zipcodes = []

for address in housing_data_reduced['full_address_or_zip_code']:
    zipcode = extract_zipcode(address)
    if zipcode:
        zipcodes.append(zipcode)
    else:
        addresses_without_zipcodes.append(address)

# Store the extracted ZIP codes in a new column
housing_data_reduced['zipcode'] = [extract_zipcode(addr) for addr in housing_data_reduced['full_address_or_zip_code']]

# Output results for verification
print("Addresses without ZIP codes:", addresses_without_zipcodes)
print("Updated DataFrame:")



Addresses without ZIP codes: ['138-35 Elder Ave,  Flushing NY, 1135', '17-30', '32-42 89th St,  E. Elmhurst NY, 1136', '34-30 78th St,  Jackson Heights NY, 1137', '35-25 77 St,  Jackson Heights NY, 1137', '61-20 Grand Central Pky,  Forest Hills NY, 1137', '78-07 Springfield Blvd,  Bayside NY, 1136', '80-35 Springfield Blvd,  Queens Village NY, 1142', '138-35 Elder Ave,  Flushing NY, 1135', '17-30', '32-42 89th St,  E. Elmhurst NY, 1136', '35-25 77 St,  Jackson Heights NY, 1137', '61-20 Grand Central Pky,  Forest Hills NY, 1137', '78-07 Springfield Blvd,  Bayside NY, 1136', '80-35 Springfield Blvd,  Queens Village NY, 1142']
Updated DataFrame:


C:\Users\lenovo\AppData\Local\Temp\ipykernel_21424\3743062092.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  housing_data_reduced['zipcode'] = [extract_zipcode(addr) for addr in housing_data_reduced['full_address_or_zip_code']]


## Handling bad input by properly formatting them and extracting zipcode. 
- Some zipcodes are null because the input in full_address_or_zip_code was badly inputted

In [634]:
# Get the indices of the rows where 'zipcode' is due to bad input
bad_inputs = housing_data_reduced[housing_data_reduced['zipcode'].isna()].index
print(bad_inputs)
# Display the indices
for index in bad_inputs:
    print(housing_data_reduced.loc[index]['full_address_or_zip_code'])


Int64Index([ 651,  803,  983, 1001, 1018, 1123, 1284, 1291, 1570, 1678, 1828,
            1855, 1938, 2072, 2078],
           dtype='int64')
138-35 Elder Ave,  Flushing NY, 1135
17-30
32-42 89th St,  E. Elmhurst NY, 1136
34-30 78th St,  Jackson Heights NY, 1137
35-25 77 St,  Jackson Heights NY, 1137
61-20 Grand Central Pky,  Forest Hills NY, 1137
78-07 Springfield Blvd,  Bayside NY, 1136
80-35 Springfield Blvd,  Queens Village NY, 1142
138-35 Elder Ave,  Flushing NY, 1135
17-30
32-42 89th St,  E. Elmhurst NY, 1136
35-25 77 St,  Jackson Heights NY, 1137
61-20 Grand Central Pky,  Forest Hills NY, 1137
78-07 Springfield Blvd,  Bayside NY, 1136
80-35 Springfield Blvd,  Queens Village NY, 1142


In [635]:
new_values = [
    "138-35 Elder Ave, Flushing NY, 11355",
    "NULL",
    "32-42 89th St, E. Elmhurst NY, 11369",
    "34-30 78th St, Jackson Heights NY, 11372",
    "35-25 77 St, Jackson Heights NY, 11372",
    "61-20 Grand Central Pky, Forest Hills NY, 11375",
    "78-07 Springfield Blvd, Bayside NY, 11364",
    "80-35 Springfield Blvd, Queens Village NY, 11427",
    "138-35 Elder Ave, Flushing NY, 11355",
    "NULL",
    "32-42 89th St, E. Elmhurst NY, 11369",
    "35-25 77 St, Jackson Heights NY, 11372",
    "61-20 Grand Central Pky, Forest Hills NY, 11375",
    "78-07 Springfield Blvd, Bayside NY, 11364",
    "80-35 Springfield Blvd, Queens Village NY, 11427"
]

for input in bad_inputs:
    print(housing_data_reduced.iloc[input])

for i in range(0, len(bad_inputs - 1)):
    housing_data_reduced.iloc[bad_inputs[i], housing_data_reduced.columns.get_loc('full_address_or_zip_code')] = new_values[i]



# # Update the 'full_address_or_zip' column for the specified indices
# housing_data_reduced.loc[bad_inputs, 'full_address_or_zip'] = new_values

# # Display the updated DataFrame
# housing_data_reduced.loc[bad_inputs]

full_address_or_zip_code    138-35 Elder Ave,  Flushing NY, 1135
model_type                                         Woodner House
date_of_sale                                                 NaT
cats_allowed                                                   0
dogs_allowed                                                   0
coop_condo                                                     2
garage_exists                                                  0
num_bedrooms                                                   2
num_floors_in_building                                      20.0
num_full_bathrooms                                             2
num_half_bathrooms                                             0
num_total_rooms                                                4
dining_room_type                                             3.0
fuel_type                                                    3.0
kitchen_type                                                 2.0
community_district_num   

In [636]:
for input in bad_inputs:
    print(housing_data_reduced.iloc[input])


full_address_or_zip_code    138-35 Elder Ave, Flushing NY, 11355
model_type                                         Woodner House
date_of_sale                                                 NaT
cats_allowed                                                   0
dogs_allowed                                                   0
coop_condo                                                     2
garage_exists                                                  0
num_bedrooms                                                   2
num_floors_in_building                                      20.0
num_full_bathrooms                                             2
num_half_bathrooms                                             0
num_total_rooms                                                4
dining_room_type                                             3.0
fuel_type                                                    3.0
kitchen_type                                                 2.0
community_district_num   

In [637]:
housing_data_reduced['zipcode'].isna().sum()

15

In [638]:
import re  # Import regular expression module

# Function to extract ZIP code
def extract_zipcode(address):
    if isinstance(address, str):
        match = re.search(r'\b\d{5}\b', address.strip())
        return match.group(0) if match else None
    return None

# Extract ZIP codes and identify addresses without ZIP codes
zipcodes = []
addresses_without_zipcodes = []

for address in housing_data_reduced['full_address_or_zip_code']:
    zipcode = extract_zipcode(address)
    if zipcode:
        zipcodes.append(zipcode)
    else:
        addresses_without_zipcodes.append(address)

# Store the extracted ZIP codes in a new column
housing_data_reduced['zipcode'] = [extract_zipcode(addr) for addr in housing_data_reduced['full_address_or_zip_code']]

# Output results for verification
print("Addresses without ZIP codes:", addresses_without_zipcodes)
print("Updated DataFrame:")


Addresses without ZIP codes: ['NULL', 'NULL']
Updated DataFrame:


C:\Users\lenovo\AppData\Local\Temp\ipykernel_21424\2648261908.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  housing_data_reduced['zipcode'] = [extract_zipcode(addr) for addr in housing_data_reduced['full_address_or_zip_code']]


In [639]:
housing_data_reduced['zipcode'].isna().sum()

2

## Mapping zipcodes to assigned zones 

In [640]:
# Define the zones mapping
zones = {
    1: [11361, 11362, 11363, 11364],
    2: [11354, 11355, 11356, 11357, 11358, 11359, 11360],
    3: [11365, 11366, 11367],
    4: [11412, 11423, 11432, 11433, 11434, 11435, 11436],
    5: [11101, 11102, 11103, 11104, 11105, 11106],
    6: [11374, 11375, 11379, 11385],
    7: [11004, 11005, 11411, 11413, 11422, 11426, 11427, 11428, 11429],
    8: [11414, 11415, 11416, 11417, 11418, 11419, 11420, 11421],
    9: [11368, 11369, 11370, 11372, 11373, 11377, 11378]
}

# Function to determine the zone based on zipcode
def determine_zone(zipcode):
    for zone, zipcodes in zones.items():
        if zipcode in zipcodes:
            return zone
    return None

# Add the zone column
housing_data_reduced['zone'] = housing_data_reduced['zipcode'].apply(lambda x: determine_zone(int(x)) if pd.notna(x) else None)


C:\Users\lenovo\AppData\Local\Temp\ipykernel_21424\2787709223.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  housing_data_reduced['zone'] = housing_data_reduced['zipcode'].apply(lambda x: determine_zone(int(x)) if pd.notna(x) else None)


In [641]:
# Assuming df is your original DataFrame
# Define the desired column order
df_order = [
    'full_address_or_zip_code', 'zipcode', 'zone', 'model_type', 'date_of_sale',
    'cats_allowed', 'dogs_allowed', 'coop_condo', 'garage_exists',
    'num_bedrooms', 'num_floors_in_building', 'num_full_bathrooms',  'num_half_bathrooms', 'num_total_rooms',
    'dining_room_type', 'fuel_type', 'kitchen_type', 
    'community_district_num',
    'approx_year_built', 'sq_footage', 'walk_score',
    'common_charges', 'maintenance_cost', 'parking_charges',
    'sale_price'
]

# Reorder the DataFrame columns
housing_data_reduced = housing_data_reduced[df_order]

# Display the rearranged DataFrame
housing_data_reduced


,full_address_or_zip_code,zipcode,zone,model_type,date_of_sale,cats_allowed,dogs_allowed,coop_condo,garage_exists,num_bedrooms,num_floors_in_building,num_full_bathrooms,num_half_bathrooms,num_total_rooms,dining_room_type,fuel_type,kitchen_type,community_district_num,approx_year_built,sq_footage,walk_score,common_charges,maintenance_cost,parking_charges,sale_price
0,"Flushing NY, 11355",11355,2.0,Mitchell Garden 3,2016-02-16,0,0,1,0,2,6.0,1,0,5,3.0,3.0,4.0,25.0,1955.0,NaN,82,NaN,NaN,0,228000.0
1,"30-11 Parsons Blvd, Flushing NY, 11354 ( Sold...",11354,2.0,Jr-4 Model,2016-02-16,0,0,1,0,1,7.0,1,0,4,5.0,1.0,4.0,25.0,1955.0,890.0,89,NaN,604.0,0,235500.0
2,"102-14 Lewis Ave, Corona NY, 11368",11368,9.0,Apt In Bldg,2016-02-17,0,0,2,0,1,1.0,1,0,3,3.0,NaN,2.0,24.0,2004.0,550.0,90,NaN,NaN,0,137550.0
3,"144-48 Roosevelt Ave, Flushing NY, 11354",11354,2.0,144-48 Roosevelt,2016-02-17,0,0,2,0,3,NaN,2,0,5,3.0,3.0,4.0,25.0,2002.0,NaN,94,NaN,NaN,0,545000.0
4,"245-27 76th Ave, Bellerose NY, 11426",11426,7.0,C-1,2016-02-18,1,1,1,0,2,2.0,1,0,4,3.0,3.0,4.0,26.0,1949.0,675.0,71,NaN,660.0,0,241700.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2225,"Not AvailableFlushing NY, 11355",11355,2.0,Colden Luxury Condo,NaT,0,0,2,0,2,7.0,1,0,4,3.0,3.0,5.0,25.0,1987.0,NaN,97,NaN,NaN,0,NaN
2226,"One Bay Club Dr, Bayside NY, 11360",11360,2.0,2 Br Deluxe,NaT,1,0,2,1,2,NaN,2,0,5,5.0,3.0,4.0,25.0,1983.0,NaN,82,NaN,NaN,99,NaN
2227,"Ridgewood NY, 11385",11385,6.0,Modern,NaT,0,0,2,0,3,NaN,2,0,6,5.0,3.0,5.0,24.0,2010.0,1500.0,96,NaN,NaN,0,NaN
2228,"Ridgewood NY, 11385",11385,6.0,Condo,NaT,0,0,2,0,3,4.0,2,0,6,5.0,3.0,5.0,24.0,2010.0,1600.0,96,NaN,NaN,0,NaN


In [642]:
housing_data_reduced[housing_data_reduced['zipcode'] == 11414]


,full_address_or_zip_code,zipcode,zone,model_type,date_of_sale,cats_allowed,dogs_allowed,coop_condo,garage_exists,num_bedrooms,num_floors_in_building,num_full_bathrooms,num_half_bathrooms,num_total_rooms,dining_room_type,fuel_type,kitchen_type,community_district_num,approx_year_built,sq_footage,walk_score,common_charges,maintenance_cost,parking_charges,sale_price


In [643]:

# Identify the indices of rows where 'zone' is NaN
no_zones = housing_data_reduced[housing_data_reduced['zone'].isna()].index.tolist()

print(len(no_zones))
print(no_zones)

housing_data_reduced.iloc[no_zones]['zipcode']

for index in no_zones:
        print(housing_data_reduced.loc[no_zones[i]]['full_address_or_zip_code'])
 

51
[110, 141, 252, 323, 351, 405, 696, 803, 937, 938, 939, 940, 941, 942, 943, 944, 947, 948, 949, 950, 951, 952, 953, 954, 955, 956, 957, 958, 959, 961, 1678, 1786, 1787, 1788, 1789, 1790, 1791, 1792, 1795, 1796, 1797, 1798, 1799, 1800, 1801, 1802, 1803, 1804, 1805, 1806, 1808]
26910 Grand Central Pky,  Floral Park NY, 11005
26910 Grand Central Pky,  Floral Park NY, 11005
26910 Grand Central Pky,  Floral Park NY, 11005
26910 Grand Central Pky,  Floral Park NY, 11005
26910 Grand Central Pky,  Floral Park NY, 11005
26910 Grand Central Pky,  Floral Park NY, 11005
26910 Grand Central Pky,  Floral Park NY, 11005
26910 Grand Central Pky,  Floral Park NY, 11005
26910 Grand Central Pky,  Floral Park NY, 11005
26910 Grand Central Pky,  Floral Park NY, 11005
26910 Grand Central Pky,  Floral Park NY, 11005
26910 Grand Central Pky,  Floral Park NY, 11005
26910 Grand Central Pky,  Floral Park NY, 11005
26910 Grand Central Pky,  Floral Park NY, 11005
26910 Grand Central Pky,  Floral Park NY, 11005


### Apparantly there are 51 entries that have null zone values because they are located in Floral Park, NY which is not in the scope of this study

In [644]:
with_zone_housing_data = housing_data_reduced.dropna(subset=['zone'])
with_zone_housing_data['zone'].isna().sum()


0

## Drop model_type, common_charges because it is too much garbage

In [645]:
with_zone_housing_data = with_zone_housing_data.drop(['model_type', 'common_charges'], axis=1)
with_zone_housing_data

,full_address_or_zip_code,zipcode,zone,date_of_sale,cats_allowed,dogs_allowed,coop_condo,garage_exists,num_bedrooms,num_floors_in_building,num_full_bathrooms,num_half_bathrooms,num_total_rooms,dining_room_type,fuel_type,kitchen_type,community_district_num,approx_year_built,sq_footage,walk_score,maintenance_cost,parking_charges,sale_price
0,"Flushing NY, 11355",11355,2.0,2016-02-16,0,0,1,0,2,6.0,1,0,5,3.0,3.0,4.0,25.0,1955.0,NaN,82,NaN,0,228000.0
1,"30-11 Parsons Blvd, Flushing NY, 11354 ( Sold...",11354,2.0,2016-02-16,0,0,1,0,1,7.0,1,0,4,5.0,1.0,4.0,25.0,1955.0,890.0,89,604.0,0,235500.0
2,"102-14 Lewis Ave, Corona NY, 11368",11368,9.0,2016-02-17,0,0,2,0,1,1.0,1,0,3,3.0,NaN,2.0,24.0,2004.0,550.0,90,NaN,0,137550.0
3,"144-48 Roosevelt Ave, Flushing NY, 11354",11354,2.0,2016-02-17,0,0,2,0,3,NaN,2,0,5,3.0,3.0,4.0,25.0,2002.0,NaN,94,NaN,0,545000.0
4,"245-27 76th Ave, Bellerose NY, 11426",11426,7.0,2016-02-18,1,1,1,0,2,2.0,1,0,4,3.0,3.0,4.0,26.0,1949.0,675.0,71,660.0,0,241700.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2225,"Not AvailableFlushing NY, 11355",11355,2.0,NaT,0,0,2,0,2,7.0,1,0,4,3.0,3.0,5.0,25.0,1987.0,NaN,97,NaN,0,NaN
2226,"One Bay Club Dr, Bayside NY, 11360",11360,2.0,NaT,1,0,2,1,2,NaN,2,0,5,5.0,3.0,4.0,25.0,1983.0,NaN,82,NaN,99,NaN
2227,"Ridgewood NY, 11385",11385,6.0,NaT,0,0,2,0,3,NaN,2,0,6,5.0,3.0,5.0,24.0,2010.0,1500.0,96,NaN,0,NaN
2228,"Ridgewood NY, 11385",11385,6.0,NaT,0,0,2,0,3,4.0,2,0,6,5.0,3.0,5.0,24.0,2010.0,1600.0,96,NaN,0,NaN


## Assumptions with date
- Range 02/2016 - 02/2017
- NA dates are in-range

In [646]:
with_zone_housing_data['date_of_sale'].count()

522

## Extract just the year

In [647]:
with_zone_housing_data['sale_year'] = with_zone_housing_data['date_of_sale'].dt.year
with_zone_housing_data

,full_address_or_zip_code,zipcode,zone,date_of_sale,cats_allowed,dogs_allowed,coop_condo,garage_exists,num_bedrooms,num_floors_in_building,num_full_bathrooms,num_half_bathrooms,num_total_rooms,dining_room_type,fuel_type,kitchen_type,community_district_num,approx_year_built,sq_footage,walk_score,maintenance_cost,parking_charges,sale_price,sale_year
0,"Flushing NY, 11355",11355,2.0,2016-02-16,0,0,1,0,2,6.0,1,0,5,3.0,3.0,4.0,25.0,1955.0,NaN,82,NaN,0,228000.0,2016.0
1,"30-11 Parsons Blvd, Flushing NY, 11354 ( Sold...",11354,2.0,2016-02-16,0,0,1,0,1,7.0,1,0,4,5.0,1.0,4.0,25.0,1955.0,890.0,89,604.0,0,235500.0,2016.0
2,"102-14 Lewis Ave, Corona NY, 11368",11368,9.0,2016-02-17,0,0,2,0,1,1.0,1,0,3,3.0,NaN,2.0,24.0,2004.0,550.0,90,NaN,0,137550.0,2016.0
3,"144-48 Roosevelt Ave, Flushing NY, 11354",11354,2.0,2016-02-17,0,0,2,0,3,NaN,2,0,5,3.0,3.0,4.0,25.0,2002.0,NaN,94,NaN,0,545000.0,2016.0
4,"245-27 76th Ave, Bellerose NY, 11426",11426,7.0,2016-02-18,1,1,1,0,2,2.0,1,0,4,3.0,3.0,4.0,26.0,1949.0,675.0,71,660.0,0,241700.0,2016.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2225,"Not AvailableFlushing NY, 11355",11355,2.0,NaT,0,0,2,0,2,7.0,1,0,4,3.0,3.0,5.0,25.0,1987.0,NaN,97,NaN,0,NaN,NaN
2226,"One Bay Club Dr, Bayside NY, 11360",11360,2.0,NaT,1,0,2,1,2,NaN,2,0,5,5.0,3.0,4.0,25.0,1983.0,NaN,82,NaN,99,NaN,NaN
2227,"Ridgewood NY, 11385",11385,6.0,NaT,0,0,2,0,3,NaN,2,0,6,5.0,3.0,5.0,24.0,2010.0,1500.0,96,NaN,0,NaN,NaN
2228,"Ridgewood NY, 11385",11385,6.0,NaT,0,0,2,0,3,4.0,2,0,6,5.0,3.0,5.0,24.0,2010.0,1600.0,96,NaN,0,NaN,NaN


In [648]:
# Assuming df is your original DataFrame
# Define the desired column order
df_order = [
    'full_address_or_zip_code', 'zipcode', 'date_of_sale', 'sale_year', 'zone',
    'cats_allowed', 'dogs_allowed', 'coop_condo', 'garage_exists',
    'num_bedrooms', 'num_floors_in_building', 'num_full_bathrooms',  'num_half_bathrooms', 'num_total_rooms',
    'dining_room_type', 'fuel_type', 'kitchen_type', 
    'community_district_num',
    'approx_year_built', 'sq_footage', 'walk_score', 'maintenance_cost',
    'parking_charges',
    'sale_price'
]

# Reorder the DataFrame columns
with_zone_housing_data = with_zone_housing_data[df_order]

# Display the rearranged DataFrame
with_zone_housing_data

,full_address_or_zip_code,zipcode,date_of_sale,sale_year,zone,cats_allowed,dogs_allowed,coop_condo,garage_exists,num_bedrooms,num_floors_in_building,num_full_bathrooms,num_half_bathrooms,num_total_rooms,dining_room_type,fuel_type,kitchen_type,community_district_num,approx_year_built,sq_footage,walk_score,maintenance_cost,parking_charges,sale_price
0,"Flushing NY, 11355",11355,2016-02-16,2016.0,2.0,0,0,1,0,2,6.0,1,0,5,3.0,3.0,4.0,25.0,1955.0,NaN,82,NaN,0,228000.0
1,"30-11 Parsons Blvd, Flushing NY, 11354 ( Sold...",11354,2016-02-16,2016.0,2.0,0,0,1,0,1,7.0,1,0,4,5.0,1.0,4.0,25.0,1955.0,890.0,89,604.0,0,235500.0
2,"102-14 Lewis Ave, Corona NY, 11368",11368,2016-02-17,2016.0,9.0,0,0,2,0,1,1.0,1,0,3,3.0,NaN,2.0,24.0,2004.0,550.0,90,NaN,0,137550.0
3,"144-48 Roosevelt Ave, Flushing NY, 11354",11354,2016-02-17,2016.0,2.0,0,0,2,0,3,NaN,2,0,5,3.0,3.0,4.0,25.0,2002.0,NaN,94,NaN,0,545000.0
4,"245-27 76th Ave, Bellerose NY, 11426",11426,2016-02-18,2016.0,7.0,1,1,1,0,2,2.0,1,0,4,3.0,3.0,4.0,26.0,1949.0,675.0,71,660.0,0,241700.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2225,"Not AvailableFlushing NY, 11355",11355,NaT,NaN,2.0,0,0,2,0,2,7.0,1,0,4,3.0,3.0,5.0,25.0,1987.0,NaN,97,NaN,0,NaN
2226,"One Bay Club Dr, Bayside NY, 11360",11360,NaT,NaN,2.0,1,0,2,1,2,NaN,2,0,5,5.0,3.0,4.0,25.0,1983.0,NaN,82,NaN,99,NaN
2227,"Ridgewood NY, 11385",11385,NaT,NaN,6.0,0,0,2,0,3,NaN,2,0,6,5.0,3.0,5.0,24.0,2010.0,1500.0,96,NaN,0,NaN
2228,"Ridgewood NY, 11385",11385,NaT,NaN,6.0,0,0,2,0,3,4.0,2,0,6,5.0,3.0,5.0,24.0,2010.0,1600.0,96,NaN,0,NaN


# Export Data as CSV

In [649]:
with_zone_housing_data.to_csv('housing_clean_imputed_data.csv', index=False)
